In [0]:
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import glob

# Make sure you have access to the images' folders
drive.mount('/content/drive/')

Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
cd drive/Team Drives

/content/drive/Team Drives


In [0]:
cd ECENproject2

/content/drive/Team Drives/ECENproject2


In [0]:
#### Importing the training images  ####
# This needs Drupad's script for corroboration
#--------------------------------------#
#images = []
#i = 0

#images_list = glob.glob("stitched_pics/*.jpg")
#images = np.array([np.array(Image.open(name)) for name in images_list])

# normalizing
#images =/ 255

# !!!! STILL NEED LABELS' VALUES !!!!
#X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.25)

In [0]:
####The Convolutional Neural Network for stitched images####
#----------------------------------------------------------#

# Still requires tweaking. Also keep it simple for faster forward\backward propagations
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape= (360, 320, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [0]:
####       Running the CNN          ####
#--------------------------------------#

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

#history = model.fit(X_train, y_train, epochs = 30, batch_size = 200, verbose = 2)

#loss  = model.evaluate(X_test, y_test)
#predictions = model.predict(X_test)

#model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 358, 318, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 179, 159, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 177, 157, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 88, 78, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 84, 74, 64)        102464    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 42, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 37, 64)        0         
__________

In [0]:
nb_epochs = 5
batch_size = 35
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.3,
    zoom_range=0.25,
    horizontal_flip=True,
    validation_split=0.05) # set validation split

train_generator = train_datagen.flow_from_directory(
    'stitched_pic',
    target_size=(360, 320),#width , height 
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    'stitched_pic', # same directory as training data
    target_size=(360, 320),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

model.save("stitched_result_1.h5")

Found 2968 images belonging to 2 classes.
Found 156 images belonging to 2 classes.
Epoch 1/5
84/84 [==============================] - 99s 1s/step - loss: 7.4802 - acc: 0.5255 - val_loss: 7.9439 - val_acc: 0.5071
Epoch 2/5
84/84 [==============================] - 94s 1s/step - loss: 7.6328 - acc: 0.5264 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 3/5
84/84 [==============================] - 92s 1s/step - loss: 7.5862 - acc: 0.5293 - val_loss: 7.9924 - val_acc: 0.5041
Epoch 4/5
84/84 [==============================] - 92s 1s/step - loss: 7.6177 - acc: 0.5274 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 5/5
84/84 [==============================] - 91s 1s/step - loss: 7.6547 - acc: 0.5251 - val_loss: 7.5928 - val_acc: 0.5289
